# K-Means vs Hierarchical Clustering vs Bayesian Hierarchical Clustering

As an unsupervised learning model, for a given set of features a clustering model tries to form groups of a given number ( usually defined as k) from the data set. 

Both **k-means** and **hierarchical clustering** use a distance metric such as Euclidan Distance or Squared Euclidian Distance to form the clusters.

**K-means**  starts with initial K-center mean points and assigns each observations to the center point with the lowest distance metric and then further updates the means based on the centroids of each assigned clusters.

**Hierarchical Clustering** model initiliazes a node for each point, finds two nodes with the lowest distance and joins these nodes untill there is one node or k nodes left. This allows us to work with a tree form structure.

**Bayesian Hierarchical Clustering** uses the same iterative process as standard hierarchical clustering but rather then using a distance metric, it uses a probability function that is calculated based on the Hypothesis that two nodes belongs to the same cluster.

# Bayesian Hierarchical Clustering

## Probability Calculations

A brief explanations for the probability calculations are provided here, for further explanation please see :http://www2.stat.duke.edu/~kheller/bhcnew.pdf and https://github.com/mtw25/Stat663FinalProject-Matt_Yamac/blob/master/Final_Project_STA_663%20(1).pdf

For each step, we find the the maximum \\(r_k\\) value among all possible combinations of nodes \\(i,j\\). The given nodes with the maximum value are then combined the create a new node. 

\\(r_k\\) value is calculated as follows:

$$r_k= \frac{\pi_k p(D_k|H_1^k)}{p(D_k|T_k)}$$ 
 

$$p(D_k|T_k) = \pi_k p(D_k|H_1^k)+(1-\pi_k)p(D_i|T_i)p(D_j|T_j) $$


$$ p(D_k|H_1^k)=\frac{1}{\pi^{nd/2}} \frac{\Gamma_d(v_n/2)}{\Gamma_d(v_0/2)} \frac{|\Lambda_0|^{v_0/2}}{|\Lambda_n|^{v_n/2}} (\frac{\kappa_0}{\kappa_n})^{d/2}  $$

where:\\(\kappa_n= \kappa_0 +n  , v_n = v_0 +n \\) and \\(\Gamma_d\\) is the  Cumilative Distribution Function of multivariate Gamma Distrubition

\\(\kappa_0 , v_0 ,\mu_0 , \Lambda_0,\alpha \\) are parameters to be initialized they will be discussed further. Specifically, \\(\kappa_0 , v_0 ,\mu_0 , \Lambda_0,\\) are the parameters of Normal Inverse Wishar prior.

We start by initializing $$\pi_i= \alpha \\ d_i=1$$ 

And update $$d_k=\alpha\Gamma(n_k)+ d_id_j$$  $$\pi_k=\alpha\frac{\Gamma(n_k)}{d_k}$$

At each iterative step \\(r_k\\) values are calculated as explained above and new clusters are formed.

# Building the Algorithm

This implementation uses a tree structure defined as Node. Each Node has the following instance variables:
- points: A set that represents each observation from the dataset.
- d: \\(d_i\\) value described above, initialized as \\(\alpha\\)
- number: Unique number that identifies each node
- left,right: Respective left and right nodes of each node.
- ph: Probability of points being in the cluster, initialize to 0
- pit, rit: Dictionaries for keeping track of \\((r_k\\) and \\(\pi_k\\)) values of each potential combinations. Note that this varibles were added for the fast version of the algorithm

In [10]:
import numpy as np
from scipy.special import factorial,multigammaln
from decimal import Decimal
import pandas as pd
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import cluster

## Node Class

In [14]:
## Define a new class called node which has the data points d_k and the number of the clusters


class Node:


    
    def __init__(self,p,alpha,i):
        
        self.single=True
        self.points=set()
        self.points.add(p)
        self.d=alpha
        self.number=i
        self.left=None
        self.right=None
        self.ph=0
## Add list that holds probability with other nodes
        self.pit={}
        self.rit={}


    def add(self,x):
        self.points.add(x)
        
    def add_all(self,x):
        self.points=x
        
    def remove(self,x):
        self.points.remove(x)
        
    def combine(self,y,alpha,i=None):
        ## Unless given a number set the number to the initial node
        p=self.points.union(y.points)
        z=Node(1,self.d,self.number)
        if i!=None:
            z.number=i
        z.left=self
        z.right=y
        z.remove(1)
        z.points=p
        z.d= alpha*factorial(len(p)-1)+self.d*y.d
        z.single=False
        return z 


## Cluster Function - Original Version
This function uses two numpy matrices to calculate and update the \\((r_k\\) and \\(\pi_k\\)) after each iteration. It is computationally really slow and has a complexity of \\(O(n^3)\\)



In [13]:
## Function that calculates the probability of the hyphothesis that the nodes belongs to the same cluster
        
def prob_hypo(X,kappa0,v0,mu0,eta0):
    from decimal import Decimal
    nf,df= X.shape
    n=Decimal(nf)
    d=Decimal(df)
    a= (1/(Decimal(np.pi))**(n*d/2))
    b=Decimal(multigammaln((v0+nf)/2,df)/multigammaln(v0/2,df))
    S=np.zeros((df,df))
    for i in range(nf):
        o=X[i]-X.mean(axis=0)
        S+=np.outer(o,o)
    etan=(eta0) + S + (kappa0*nf/(kappa0+nf))*np.outer((X.mean(axis=0)-mu0),(X.mean(axis=0)-mu0))
    c=Decimal(np.linalg.det(eta0)**(v0/2))/(Decimal(np.linalg.det(etan))**((Decimal(v0)+n)/2))
    d=Decimal(kappa0/(kappa0+nf))**(d/2)
    return float(a*b*c*d)

## Function that initiates clusters from the given data set and the parameters for the prior
def init(X,alpha,kappa0,v0,mu0,eta0):
    x=[]
    for i in range(len(X)):
        node=Node(i,alpha,i)
        node.ph=prob_hypo(X[[i]],kappa0,v0,mu0,eta0)
        
        x.append(node)
        
    return x


## Function that calculates P and R values between each nodes and puts them into two numpy matrices
## @ nodes,list of nodes taken from the data set, X = data set

def calculate_r(nodes,alpha,X,kappa0,v0,mu0,eta0):
    from scipy.special import factorial
    n=len(nodes)
    rik=np.zeros((n,n))
    pit=np.zeros((n,n))
    for i in range(n):
        for j in range(i+1,n):
            pit[i,j],rik[i,j]= get_pi_ri(nodes[i],nodes[j],alpha)
            
    np.fill_diagonal(rik,-10)
    return rik,pit

## Function for updating clusters, by creating a new node (combination of i and j) and deleting nodes i,j
def update_clust(rk,pit,nodes,alpha):
    i,j=np.unravel_index(np.argmax(rk),rk.shape)
    if len(nodes[i].points)>len(nodes[j].points):
        new_node=nodes[i].combine(nodes[j],alpha)
    else:
        new_node=nodes[j].combine(nodes[i],alpha)
    new_node.single=False
    new_node.ph=pit[i,j]
    nodes[i]=new_node 
    del nodes[j]
    return nodes


## Final function, returns y_hat and the root of the tree
def clust(X,alpha,mu0,eta0,kappa0=1,v0=2.5,k=3):
    nodes=init(X,alpha,kappa0,v0,mu0,eta0)
   ## Loop until n-k and set the cluster
    for i in range(len(nodes)-k):
        rk,pit=calculate_r(nodes,alpha,X,kappa0, v0,mu0, eta0)
        nodes=update_clust(rk,pit,nodes,alpha)
    y=np.zeros(len(X),dtype="int32")
    for i in range(k):
        ind=list(nodes[i].points)
        y[ind]=i
    ## Finish the loop to output the final root 
    for i in range(k-1):
        rk,pit=calculate_r(nodes,alpha,X,kappa0,v0,mu0,eta0)
        nodes=update_clust(rk,pit,nodes,alpha)
        
    return y,nodes[0]

## Cluster Function-Fast Version

This version replaces the numpy matrcies with two dictionaries defined for each node.Rather creating a numpy matrix after updating the nodes, this version calculates the initial \\((r_k\\) , \\(\pi_k\\)) for each point and keeps updating them after forming a new cluster. This results in a faster and efficient calculation. The run time complexity is reduced from \\(O(n^3)\\) to \\(O(n^2)\\)

In [16]:
## Function that calculates the probability of points being in the same cluster
        
def prob_hypo(X,kappa0,v0,mu0,eta0):
    from decimal import Decimal
    nf,df= X.shape
    n=Decimal(nf)
    d=Decimal(df)
    a= (1/(Decimal(np.pi))**(n*d/2))
    b=Decimal(multigammaln((v0+nf)/2,df)/multigammaln(v0/2,df))
    S=np.zeros((df,df))
    for i in range(nf):
        o=X[i]-X.mean(axis=0)
        S+=np.outer(o,o)
    etan=(eta0) + S + (kappa0*nf/(kappa0+nf))*np.outer((X.mean(axis=0)-mu0),(X.mean(axis=0)-mu0))
    c=Decimal(np.linalg.det(eta0)**(v0/2))/(Decimal(np.linalg.det(etan))**((Decimal(v0)+n)/2))
    d=Decimal(kappa0/(kappa0+nf))**(d/2)
    return float(a*b*c*d)

## PIT, RIT calculatios between two nodes
def get_pi_ri(i,j,alpha):
    clust_k=i.combine(j,alpha)
    nk=len(clust_k.points)
    dk=clust_k.d
    pi=alpha*factorial(nk-1)/dk 
    all_points=list(clust_k.points)
    ph=prob_hypo(X[all_points][:],kappa0,v0,mu0,eta0) 
    pit=ph*pi+ (1-pi)*i.ph*j.ph
    rit=(pi*ph)/pit
    return pit,rit
    
    
## Function that calculates the maximum value in a dictionary, return key and max value
def get_dict_max(d):
    ind=0
    m=0
    for i in d:
        if d[i]>=m:
            m=d[i]
            ind=i
    return ind,m

## Get Node with the specific number from a list of nodes

def get_node(i,nodes):
    for node in nodes:
        if node.number==i:
            return node
    
## Incorporate pi,ri calculations go over all the nodes one inital time
def init2(X,alpha,kappa0,v0,mu0,eta0):
    x=[]
    for i in range(len(X)):
        node=Node(i,alpha,i)
        node.ph=prob_hypo(X[[i]],kappa0,v0,mu0,eta0)
        
        x.append(node)
    
    from scipy.special import factorial
    n=len(x)
    for i in range(n):
        for j in range(i+1,n):
            p,r=get_pi_ri(x[i],x[j],alpha)
            x[i].pit[j]=p
            x[i].rit[j]=r  
        
        
    return x

## Function that creates a new node and drops the nodes that give the highest pit value. Add the pit,ri values to the new node

def change_nodes(i,j,n,nodes,alpha):
 
    n1=get_node(i,nodes)
    n2=get_node(j,nodes)

    new_node=n1.combine(n2,alpha,i=n)
    new_node.ph=get_pi_ri(n1,n2,alpha=alpha)[0] 
    nodes.remove(n1)
    nodes.remove(n2)
    for node in nodes:
        
        new_node.pit[node.number],new_node.rit[node.number]=get_pi_ri(node,new_node,alpha=alpha)   
        ## nodes[k].pit[n],nodes[k].rit[n] =get_pi_ri(nodes[k],new_node,alpha=alpha) 
        if n1.number in node.rit:
            del node.rit[n1.number]
        if  n2.number in node.rit:
            del node.rit[n2.number]
    nodes.append(new_node)
    return nodes 

## Function for updating clusters, by creating a new node (combination of i and j) and deleting nodes i,j 
## and updating the probability values of each dictionary
def update_clust2(nodes,n,alpha):    
    ## Find the maximum rit
    m=0
    i=0
    j=0
    for k in range(len(nodes)):
        cind,cm=get_dict_max(nodes[k].rit)
        if cm>=m:
            m=cm
            i=nodes[k].number
            j=cind
    return change_nodes(i,j,n,nodes,alpha=alpha)
 

def clust2(X,alpha,mu0,eta0,kappa0=1,v0=2.5,k=3):
    n=len(X)
    nodes=init2(X,alpha,kappa0,v0,mu0,eta0)
    y=np.zeros((n))
    for i in range(n-k):
        nodes=update_clust2(nodes,n+i,alpha)
    ## Set the labels
    y=np.zeros(len(X),dtype="int32")
    for i in range(k):
        ind=list(nodes[i].points)
        y[ind]=i
    ## Finish the classification and produce the table
    for i in range(k-1) :
        nodes=update_clust2(nodes,n+i,alpha)
    return y,nodes